In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import scanpy as sc
import jax
import os
from cfp.metrics import compute_metrics, compute_mean_metrics, compute_metrics_fast
import cfp.preprocessing as cfpp
import anndata as ad
import pandas as pd
from tqdm.auto import tqdm
import numpy as np

/home/haicu/soeren.becker/miniconda3/envs/env_cfp2/lib/python3.12/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/home/haicu/soeren.becker/miniconda3/envs/env_cfp2/lib/python3.12/site-packages/anndata/utils.py:429: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/home/haicu/soeren.becker/miniconda3/envs/env_cfp2/lib/python3.12/site-packages/anndata/utils.py:429: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
/home/haicu/soeren.becker/miniconda3/envs/env_cfp2/lib/python3.12/site-packages/anndata/utils.py:429: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
/home/haicu/soeren.becker/miniconda3/envs/

In [3]:
def get_mask(x, y):
    return x[:, [gene in y for gene in adata_train.var_names]]

In [4]:
split = 4

### Load data

In [5]:
DATA_DIR = "/home/haicu/soeren.becker/repos/ot_pert_reproducibility/norman2019/norman_preprocessed_adata"

adata_train_path = os.path.join(DATA_DIR, f"adata_train_pca_50_split_{split}.h5ad")
adata_test_path = os.path.join(DATA_DIR, f"adata_val_pca_50_split_{split}.h5ad")
adata_ood_path = os.path.join(DATA_DIR, f"adata_test_pca_50_split_{split}.h5ad")

# load data splits
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)

### Categorize perturbations into subgroups: single, double_seen_0, double_seen_1, double_seen_2

In [6]:
train_conditions = adata_train.obs.condition.str.replace("+ctrl", "").str.replace("ctrl+", "").unique()

# make sure that non of the test conditions is in the training data
assert not adata_ood[adata_ood.obs.condition != "ctrl"].obs.condition.isin(train_conditions).any()

# single perturbations
mask_single_perturbation = adata_ood.obs.condition.str.contains("ctrl")

# double_seen_0 perturbations
mask_double_perturbation_seen_0 = (
    ~adata_ood.obs.condition.str.contains("ctrl") & 
    ~adata_ood.obs.gene_1.isin(train_conditions) & 
    ~adata_ood.obs.gene_2.isin(train_conditions)
)

# double_seen_1 perturbations
mask_double_perturbation_seen_1 = (
    ~adata_ood.obs.condition.str.contains("ctrl") & 
    (
        (adata_ood.obs.gene_1.isin(train_conditions) & ~adata_ood.obs.gene_2.isin(train_conditions)) | 
        (~adata_ood.obs.gene_1.isin(train_conditions) & adata_ood.obs.gene_2.isin(train_conditions))
    )
)

# double_seen_2 perturbations
mask_double_perturbation_seen_2 = (
    ~adata_ood.obs.condition.str.contains("ctrl") & 
    adata_ood.obs.gene_1.isin(train_conditions) & 
    adata_ood.obs.gene_2.isin(train_conditions)
)

# add perturbation subgroup to anndata
adata_ood.obs.loc[mask_single_perturbation, "subgroup"] = "single"
adata_ood.obs.loc[mask_double_perturbation_seen_0, "subgroup"] = "double_seen_0"
adata_ood.obs.loc[mask_double_perturbation_seen_1, "subgroup"] = "double_seen_1"
adata_ood.obs.loc[mask_double_perturbation_seen_2, "subgroup"] = "double_seen_2"

display(adata_ood.obs.subgroup.value_counts())

subgroup
double_seen_1    14554
single           13520
double_seen_2     3934
double_seen_0     1587
Name: count, dtype: int64

In [7]:
# compute pca on full dataset
adata_all = ad.concat((adata_train, adata_test, adata_ood))
cfpp.centered_pca(adata_all, n_comps=10)

/home/haicu/soeren.becker/miniconda3/envs/env_cfp2/lib/python3.12/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [8]:
adata_train.obs

,condition,cell_type,dose_val,control,condition_name,cell_line,gene_1,gene_2,num_control,kategory
cell_barcode,,,,,,,,,,
AAACCTGCACGAAGCA-1,ctrl,A549,1,True,A549_ctrl_1,A549,ctrl,ctrl,2,ctrl
AAACCTGCAGCCTTGG-1,MAML2+ctrl,A549,1+1,False,A549_MAML2+ctrl_1+1,A549,MAML2,ctrl,1,single
AAACCTGCATCTCCCA-1,ctrl+CEBPE,A549,1+1,False,A549_ctrl+CEBPE_1+1,A549,ctrl,CEBPE,1,single
AAACCTGGTCCATCCT-1,TGFBR2+ETS2,A549,1+1,False,A549_TGFBR2+ETS2_1+1,A549,TGFBR2,ETS2,0,double
AAACCTGGTCTGATTG-1,DUSP9+ctrl,A549,1+1,False,A549_DUSP9+ctrl_1+1,A549,DUSP9,ctrl,1,single
...,...,...,...,...,...,...,...,...,...,...
TTTGTCAGTAGGCATG-8,COL2A1+ctrl,A549,1+1,False,A549_COL2A1+ctrl_1+1,A549,COL2A1,ctrl,1,single
TTTGTCAGTCACTTCC-8,ETS2+CEBPE,A549,1+1,False,A549_ETS2+CEBPE_1+1,A549,ETS2,CEBPE,0,double
TTTGTCAGTCAGAATA-8,ctrl,A549,1,True,A549_ctrl_1,A549,ctrl,ctrl,2,ctrl


In [9]:
# control cells used to make predictions
adata_train_ctrl = adata_train[adata_train.obs.condition == "ctrl"].copy()

# perturbed cells used to create additive model predictions
adata_train_single = adata_train[adata_train.obs.kategory == "single"].copy()

# conditions that the additive model can predict
adata_ood_double_seen_2 = adata_ood[adata_ood.obs.subgroup == "double_seen_2"].copy()

NUM_SAMPLED_CELLS = 500

### make sure that the perturbed gene is always gene 1

In [10]:
genes = adata_train_single.obs.condition.str.split("+", expand=True).values
genes_1 = genes[:, 0]
genes_2 = genes[:, 1]
mask = genes_1 == "ctrl"

genes_1[mask], genes_2[mask] = genes_2[mask], genes_1[mask]

adata_train_single.obs.loc[:, "condition_ordered"] = genes_1 + "+" + genes_2
adata_train_single.obs.loc[:, "gene_1_ordered"] = genes_1 # this is the perturbed gene
adata_train_single.obs.loc[:, "gene_2_ordered"] = genes_2 # this is always ctrl

assert np.all(adata_train_single.obs.loc[:, "gene_1_ordered"] != "ctrl")
assert np.all(adata_train_single.obs.loc[:, "gene_2_ordered"] == "ctrl")

In [11]:
display(adata_train_single.obs)

,condition,cell_type,dose_val,control,condition_name,cell_line,gene_1,gene_2,num_control,kategory,condition_ordered,gene_1_ordered,gene_2_ordered
cell_barcode,,,,,,,,,,,,,
AAACCTGCAGCCTTGG-1,MAML2+ctrl,A549,1+1,False,A549_MAML2+ctrl_1+1,A549,MAML2,ctrl,1,single,MAML2+ctrl,MAML2,ctrl
AAACCTGCATCTCCCA-1,ctrl+CEBPE,A549,1+1,False,A549_ctrl+CEBPE_1+1,A549,ctrl,CEBPE,1,single,CEBPE+ctrl,CEBPE,ctrl
AAACCTGGTCTGATTG-1,DUSP9+ctrl,A549,1+1,False,A549_DUSP9+ctrl_1+1,A549,DUSP9,ctrl,1,single,DUSP9+ctrl,DUSP9,ctrl
AAACCTGTCAGGCGAA-1,ctrl+FOXA1,A549,1+1,False,A549_ctrl+FOXA1_1+1,A549,ctrl,FOXA1,1,single,FOXA1+ctrl,FOXA1,ctrl
AAACCTGTCCGTAGTA-1,BCORL1+ctrl,A549,1+1,False,A549_BCORL1+ctrl_1+1,A549,BCORL1,ctrl,1,single,BCORL1+ctrl,BCORL1,ctrl
...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTCACAGTAACGG-8,MAP4K3+ctrl,A549,1+1,False,A549_MAP4K3+ctrl_1+1,A549,MAP4K3,ctrl,1,single,MAP4K3+ctrl,MAP4K3,ctrl
TTTGTCACATCGTCGG-8,ctrl+MAP2K6,A549,1+1,False,A549_ctrl+MAP2K6_1+1,A549,ctrl,MAP2K6,1,single,MAP2K6+ctrl,MAP2K6,ctrl
TTTGTCAGTAGGCATG-8,COL2A1+ctrl,A549,1+1,False,A549_COL2A1+ctrl_1+1,A549,COL2A1,ctrl,1,single,COL2A1+ctrl,COL2A1,ctrl


### Get predictions - recall that this additive model can only make predictions for subgroup double_seen_2

In [12]:
all_predictions, all_conditions = [], []

for condition in tqdm(adata_ood_double_seen_2.obs.condition.unique()):

    # get perturbed genes
    gene_1, gene_2 = condition.split("+")

    # get perturbed gene expression profiles
    cells_1 = np.asarray(adata_train_single[adata_train_single.obs.gene_1_ordered == gene_1].X.todense()) # gene_1_ordered is always the perturbed gene
    cells_2 = np.asarray(adata_train_single[adata_train_single.obs.gene_1_ordered == gene_2].X.todense())

    # get control cells
    random_idcs_ctrl = np.random.choice(adata_train_ctrl.shape[0], size=NUM_SAMPLED_CELLS, replace=True)
    ctrl_cells = np.asarray(adata_train_ctrl.X[random_idcs_ctrl].todense())

    # compute displacements
    displacement_1 = cells_1.mean(axis=0) - ctrl_cells.mean(axis=0)
    displacement_2 = cells_2.mean(axis=0) - ctrl_cells.mean(axis=0)

    # get predictions
    predictions = ctrl_cells + displacement_1 + displacement_2
    all_predictions.append(predictions)
    all_conditions.extend([condition] * NUM_SAMPLED_CELLS)

  0%|          | 0/13 [00:00<?, ?it/s]

In [13]:
adata_pred_ood  = ad.AnnData(X=np.vstack(all_predictions), obs=pd.DataFrame(all_conditions, columns=["condition"]))
adata_pred_ood

/home/haicu/soeren.becker/miniconda3/envs/env_cfp2/lib/python3.12/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


AnnData object with n_obs × n_vars = 6500 × 5045
    obs: 'condition'

### create dicts with predictions and ground truths per condition

In [14]:
cfpp.project_pca(query_adata=adata_pred_ood, ref_adata=adata_all)
cfpp.project_pca(query_adata=adata_ood_double_seen_2, ref_adata=adata_all)

ood_data_target_encoded, ood_data_target_decoded = {}, {}
ood_data_target_encoded_predicted, ood_data_target_decoded_predicted = {}, {}

for cond in tqdm(adata_ood_double_seen_2.obs["condition"].cat.categories):
    if cond == "ctrl":
        continue
    
    # get masks for truths and preds for this condition
    select = adata_ood_double_seen_2.obs["condition"] == cond
    select2 = adata_pred_ood.obs["condition"] == cond
    
    # truth and preds in pca space
    ood_data_target_encoded[cond] = adata_ood_double_seen_2[select].obsm["X_pca"]
    ood_data_target_encoded_predicted[cond] = adata_pred_ood[select2].obsm["X_pca"]

    # truths and preds in gene space
    ood_data_target_decoded[cond] = np.asarray(adata_ood_double_seen_2[select].X.todense())
    ood_data_target_decoded_predicted[cond] = adata_pred_ood[select2].X

  0%|          | 0/13 [00:00<?, ?it/s]

In [15]:
ood_metrics_encoded, mean_ood_metrics_encoded = {}, {}
ood_metrics_decoded, mean_ood_metrics_decoded = {}, {}
deg_ood_metrics, deg_mean_ood_metrics = {}, {}
ood_deg_dict = {}
ood_deg_target_decoded_predicted, ood_deg_target_decoded = {}, {}

print("Computing ood_metrics_encoded")
# ood set: evaluation in encoded (=pca) space
ood_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, 
    # compute_metrics_fast, 
    ood_data_target_encoded, 
    ood_data_target_encoded_predicted
)
mean_ood_metrics_encoded = compute_mean_metrics(
    ood_metrics_encoded, 
    prefix="encoded_ood_",
)

print("Computing ood_metrics_decoded")
# ood set: evaluation in decoded (=gene) space
ood_metrics_decoded = jax.tree_util.tree_map(
    # compute_metrics, 
    compute_metrics_fast, 
    ood_data_target_decoded, 
    ood_data_target_decoded_predicted
)
mean_ood_metrics_decoded = compute_mean_metrics(
    ood_metrics_decoded, 
    prefix="decoded_ood_",
)

# ood set
ood_deg_dict = {
    k: v
    for k, v in adata_train.uns['rank_genes_groups_cov_all'].items() 
    if k in ood_data_target_decoded_predicted.keys()
}

print("Apply DEG mask")
# ood set
ood_deg_target_decoded_predicted = jax.tree_util.tree_map(
    get_mask, 
    ood_data_target_decoded_predicted, 
    ood_deg_dict
)

ood_deg_target_decoded = jax.tree_util.tree_map(
    get_mask, 
    ood_data_target_decoded, 
    ood_deg_dict
)

print("Compute metrics on DEG subsetted decoded")
deg_ood_metrics = jax.tree_util.tree_map(
    compute_metrics, 
    # compute_metrics_fast, 
    ood_deg_target_decoded, 
    ood_deg_target_decoded_predicted
)
deg_mean_ood_metrics = compute_mean_metrics(
    deg_ood_metrics, 
    prefix="deg_ood_"
)

Computing ood_metrics_encoded
Computing ood_metrics_decoded
Apply DEG mask
Compute metrics on DEG subsetted decoded


In [16]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.9778423171776992,
 'deg_ood_sinkhorn_div_1': 13.68984838632437,
 'deg_ood_sinkhorn_div_10': 3.2712959142831655,
 'deg_ood_sinkhorn_div_100': 1.450524696936974,
 'deg_ood_e_distance': np.float64(2.5361580523698413),
 'deg_ood_mmd': np.float32(0.018801868)}

In [17]:
collected_results = {
    "ood_metrics_encoded": ood_metrics_encoded,
    "mean_ood_metrics_encoded": mean_ood_metrics_encoded,
    "ood_metrics_decoded": ood_metrics_decoded,
    "mean_ood_metrics_decoded": mean_ood_metrics_decoded,
    "deg_ood_metrics": deg_ood_metrics,
    "deg_mean_ood_metrics": deg_mean_ood_metrics,
    "ood_deg_dict": ood_deg_dict,
    "ood_deg_target_decoded_predicted": ood_deg_target_decoded_predicted,
    "ood_deg_target_decoded": ood_deg_target_decoded,
}

In [18]:
OUT_DIR = "/lustre/groups/ml01/workspace/ot_perturbation/data/norman_soren/additive_mean_displacement/num_samples_500"
os.makedirs(OUT_DIR, exist_ok=True)
pd.to_pickle(collected_results, os.path.join(OUT_DIR, f"norman_additive_split_{split}_collected_results.pkl"))